In [6]:

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt 
import time
from src.utils.cp_utils import * 
from src.models.conformal_model import ConformalModel, ConformalModelLogits
import torch.backends.cudnn as cudnn
import random
from src.data.make_cifar10_dataset import CIFAR10, get_img_transformer
from src.models.cifar10_conv_model import Cifar10ConvModel

from src.utils.bootstrap_utils import *

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataset = CIFAR10(split="train", root='../data/processed', download=True,
                      transform=get_img_transformer())
test_dataset = CIFAR10(split="test", root='../data/processed', download=True,
                      transform=get_img_transformer())
test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=32)
calib_dataset = CIFAR10(split="calib", root='../data/processed', download=True,
                      transform=get_img_transformer())
calib_dataloader = torch.utils.data.DataLoader(
        calib_dataset, batch_size=32)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [50]:
model1 = Cifar10ConvModel()
model1.load_state_dict(torch.load("../models/trained_model.pt"))
model1.eval()
model1 = torch.nn.DataParallel(model1).cpu()
model2 = Cifar10ConvModel()
model2.load_state_dict(torch.load("../models/trained_model.pt"))
model2.eval()
model2 = torch.nn.DataParallel(model2).cpu()
model3 = Cifar10ConvModel()
model3.load_state_dict(torch.load("../models/trained_model.pt"))
model3.eval()
model3 = torch.nn.DataParallel(model3).cpu()

In [54]:
models = [model1, model2, model3]
train_ensemble(models, train_dataset, print_acc=True)

Step 500     training accuracy: 0.7320625
Step 1000    training accuracy: 0.77625
Step 1500    training accuracy: 0.7749107142857142
Step 2000    training accuracy: 0.806171875
Step 2500    training accuracy: 0.8296875
Step 3000    training accuracy: 0.8255208333333334
Step 3500    training accuracy: 0.8332291666666667
Step 4000    training accuracy: 0.8415234375
Step 4500    training accuracy: 0.8605
Step 5000    training accuracy: 0.8684375
Step 5500    training accuracy: 0.8669196428571428
Step 6000    training accuracy: 0.88453125
Step 6500    training accuracy: 0.904375
Step 7000    training accuracy: 0.9075520833333334
Step 7500    training accuracy: 0.9140625
Step 8000    training accuracy: 0.9146484375
Step 500     training accuracy: 0.7211875
Step 1000    training accuracy: 0.75859375
Step 1500    training accuracy: 0.7598214285714285
Step 2000    training accuracy: 0.79390625
Step 2500    training accuracy: 0.8346875
Step 3000    training accuracy: 0.81578125
Step 3500    tra

In [61]:
models = [model1, model2, model3]
for inputs, targets in  test_dataloader:
    with torch.no_grad():
        output = get_ensemble_preparation(models, inputs[0])
        probs = torch.stack([nnf.softmax(x, dim=1) for x in output])
        y1 = [r[:, 0].item() for r in probs]
        y3 = [r[:, 3].item() for r in probs]
        print(probs)
        print(np.std(y1))
        print(np.std(y3))
        break

tensor([[[8.1698e-08, 3.7372e-09, 3.4974e-06, 3.2972e-03, 2.6616e-10,
          9.9670e-01, 1.8103e-07, 4.9945e-08, 7.8724e-08, 1.4190e-08]],

        [[1.5091e-08, 2.4493e-06, 3.2369e-05, 9.6779e-01, 3.3871e-06,
          3.1482e-02, 2.1535e-04, 2.0943e-04, 1.8057e-05, 2.4451e-04]],

        [[1.4179e-06, 1.4989e-06, 9.7452e-04, 9.8571e-01, 1.8647e-04,
          5.2205e-03, 6.4298e-03, 1.1243e-04, 5.2094e-06, 1.3585e-03]]])
6.461454251862864e-07
0.458948905909294
